In [ ]:
import meep as mp
import numpy as np
from typing import List

# set data for wavelengt and n k materials
wavelengths = dataNK["Wavelength (nm)"]
wavelengthsUm = [i / 1000 for i in wavelengths]
auN = dataNK["Au_n"]
auK = dataNK["Au_k"]
spiroN = dataNK["Spiro_n"]
spiroK = dataNK["Spiro_k"]
perovN = dataNK["Perov_n"]
perovK = dataNK["Perov_k"]
tio2N = dataNK["TiO2_n"]
tio2K = dataNK["TiO2_k"]

# Define the dimensions of the simulation
# 1um = 1000 nm
auHeight = 0.1  # 0.1 um or 100 nm
spiroHeight = 0.2  # 0.2 um or 200 nm
perovHeight = 0.37  # 0.37 um or 370 nm
tio2Height = 0.07  # 0.07 um or 70 nm
xLength = 1  # 1 um or 1000 nm
yLength = 1  # 1 um or 1000 nm
cellHeight = auHeight + spiroHeight + perovHeight + tio2Height
cellLength = 1
cellWidth = 1
cellSim = mp.Vector3(16, 8 + 0.5)
nmtoum = 1000


# Define function to get mediums
def getMediums(wavelengths: List[float], nArray: List[float], kArray: List[float]):
    mediums = []
    for i in range(len(wavelengths)):
        Nindex = complex(nArray[i], kArray[i])
        eps = np.square(Nindex)

        D_conductivity = 2 * np.pi * (1 / (wavelengths[i])) * eps.imag / eps.real
        # print(eps.real, D_conductivity)
        med = mp.Medium(epsilon=eps.real, D_conductivity=D_conductivity)

        mediums.append(med)
        # print(vars(mediums[i]))

    return mediums


# Init list of mediums
auMat = []
spiroMat = []
perovMat = []
tio2Mat = []

# Define materials
auMat = getMediums(wavelengthsUm, auN, auK)
spiroMat = getMediums(wavelengthsUm, spiroN, spiroK)
perovMat = getMediums(wavelengthsUm, perovN, perovK)
tio2Mat = getMediums(wavelengthsUm, tio2N, tio2K)


# Define function to set geometry
def SetGeometry(index):
    geometry = [
        mp.Block(
        mp.Vector3(mp.inf, 1, mp.inf),
        center=mp.Vector3(),
        material=mp.Medium(epsilon=12),
    )
        # mp.Block(mp.Vector3(mp.inf, 1, mp.inf), material=auMat[index]),
        # mp.Block(mp.Vector3(xLength, spiroHeight, mp.inf), material=spiroMat[index])
    ]
    
    return geometry


geometryAu0 = SetGeometry(0)

# Define the central frequency and frequency width (in PHz)
f_central = 1.5  # corresponds to approximately 800nm

# Define the source
sources = [mp.Source(
    src=mp.ContinuousSource(frequency=0.5),
    component=mp.Ez,  # Assuming Ez polarization, change as needed
    center=mp.Vector3(-7, 0),  # Source position, adjust as needed
    # size=mp.Vector3(0.01, 0.01, 0),  # Source size, adjust as needed
)]

pmlLayers = [mp.PML(1)]
resolution = 10

sim = mp.Simulation(
    cell_size=cellSim,
    boundary_layers=pmlLayers,
    geometry=geometryAu0,
    sources=sources,
    resolution=resolution,
)

from matplotlib import pyplot as plt

%matplotlib inline
plt.figure(dpi=100)
sim.plot2D()
plt.show()

In [ ]:
sim.run(until=200)
plt.figure(dpi=100)
sim.plot2D(fields=mp.Ez)
plt.show()

In [ ]:
sim.reset_meep()
f = plt.figure(dpi=100)
Animate = mp.Animate2D(fields=mp.Ez, f=f, realtime=False, normalize=True)
plt.close()
sim.run(mp.at_every(1, Animate), until=100)
plt.close()

In [ ]:
filename = "media/straight_waveguide.mp4"
Animate.to_mp4(10, filename)